In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('datasets/stocks/DJIA.csv')
df
stocks = df.pivot('Date', 'Name')
print(stocks.head(10))
print(list(stocks.columns.values))

             Open                                                          \
Name         AABA   AAPL   AMZN    AXP     BA    CAT   CSCO    CVX    DIS   
Date                                                                        
2006-01-03  39.69  10.34  47.47  51.70  70.40  57.87  17.21  57.25  24.08   
2006-01-04  41.22  10.73  47.48  52.44  70.08  57.95  17.48  59.08  24.50   
2006-01-05  40.93  10.69  47.16  51.90  70.55  59.02  17.94  58.68  24.13   
2006-01-06  42.88  10.75  47.97  52.64  70.34  59.47  18.51  58.95  24.65   
2006-01-09  43.10  10.96  46.55  52.82  69.28  60.45  18.97  59.49  24.85   
2006-01-10  42.96  10.89  46.41  53.58  68.80  61.35  18.87  58.76  25.02   
2006-01-11  42.19  11.98  45.65  53.70  69.97  60.87  18.98  59.22  25.38   
2006-01-12  41.92  12.14  44.79  53.58  70.25  61.23  19.27  60.00  25.65   
2006-01-13  41.00  12.14  44.48  53.70  69.64  62.02  19.13  59.89  25.61   
2006-01-17  39.09  12.24  43.95  53.08  69.07  61.69  18.70  60.90  25.83   

In [3]:
stocks[('High', 'MMM')].head()

Date
2006-01-03    79.35
2006-01-04    79.49
2006-01-05    78.65
2006-01-06    78.90
2006-01-09    79.83
Name: (High, MMM), dtype: float64

In [4]:
# Create variables for lead in 7 days:
target  = stocks[('High', 'MMM')].shift(-7)
features =  stocks.values
target = target.values

In [5]:
target[-10:-1]

array([236.54, 237.08, 237.31,    nan,    nan,    nan,    nan,    nan,
          nan])

In [6]:
np.argwhere(np.isnan(target))

array([[3013],
       [3014],
       [3015],
       [3016],
       [3017],
       [3018],
       [3019]], dtype=int64)

In [7]:
np.argwhere(np.isnan(features))

array([[ 907,   20],
       [ 907,   51],
       [ 907,   82],
       [ 907,  113],
       [ 907,  144],
       [1068,    0],
       [1068,    1],
       [1068,    2],
       [1068,    6],
       [1068,   10],
       [1068,   14],
       [1068,   21],
       [1068,   31],
       [1068,   32],
       [1068,   33],
       [1068,   37],
       [1068,   41],
       [1068,   45],
       [1068,   52],
       [1068,   62],
       [1068,   63],
       [1068,   64],
       [1068,   68],
       [1068,   72],
       [1068,   76],
       [1068,   83],
       [1068,   93],
       [1068,   94],
       [1068,   95],
       [1068,   99],
       [1068,  103],
       [1068,  107],
       [1068,  114],
       [1068,  124],
       [1068,  125],
       [1068,  126],
       [1068,  130],
       [1068,  134],
       [1068,  138],
       [1068,  145],
       [1657,    8],
       [1657,   39],
       [1657,   70],
       [2913,    3],
       [2913,    4],
       [2913,    5],
       [2913,    7],
       [2913,

In [8]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='mean')
X_imp = imp.fit_transform(features)   

In [9]:
# Normilize data, drop NaN obs
X_imp = X_imp[:-8,]
target = target[:-8,]
import sklearn.preprocessing
X_norm = sklearn.preprocessing.normalize(X_imp)

In [10]:
stocks[('High', 'MMM')].head()

Date
2006-01-03    79.35
2006-01-04    79.49
2006-01-05    78.65
2006-01-06    78.90
2006-01-09    79.83
Name: (High, MMM), dtype: float64

In [11]:
target[1:7]

array([78.2 , 77.62, 77.28, 77.37, 76.91, 76.23])

In [12]:
stocks[('High', 'MMM')][8:15]

Date
2006-01-13    78.20
2006-01-17    77.62
2006-01-18    77.28
2006-01-19    77.37
2006-01-20    76.91
2006-01-23    76.23
2006-01-24    75.60
Name: (High, MMM), dtype: float64

# Problem 1. 
Scale the data. Use 10000 iterations with 30 steps to predict target using RNN with 3 neural layers with 100 neurons each [100,100,100].Use learning_rate = 0.001.

a. Generate predicted price from the model. 
b. Calculate expected profit from 7 day investement
c. Find 10 days with best expected rate of 7 day investement
d. calculate expected and observed profit for these 10 best investement days
e. Find average expected and realized profit for all days in the data

In [13]:
import tensorflow as tf
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()


n_steps = 30
n_inputs = len(stocks.columns)
n_neurons = 50
n_outputs = 1
learning_rate = 0.001
batch_size = 1


X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])



n_neurons = 100
n_layers = 3

layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
          for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
rnn_outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)
init = tf.global_variables_initializer()




#cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
#rnn_outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
learning_rate = 0.001
# Get output from reshaping the neurons
stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons])
stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs)
outputs = tf.reshape(stacked_outputs, [batch_size, n_steps, n_outputs])
loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [26]:
np.arange(0., n_steps)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29.])

In [24]:
def next_batch(X, y, batch_size, n_steps):
    lenst = len(X) - n_steps
    ns = batch_size // n_steps
    t0 = np.round(np.random.rand(batch_size, 1) * lenst)
    print(t0)
    Ts = t0.astype(int) + np.arange(0., n_steps)
    print(Ts)
    Xs = X[Ts.astype(int)]
    ys =y[Ts.astype(int)] 
    ys = ys.reshape(batch_size, n_steps, n_outputs)
    return Xs, ys



In [25]:
next_batch(X_norm, target, 1, 30)

[[1793.]]
[[1793. 1794. 1795. 1796. 1797. 1798. 1799. 1800. 1801. 1802. 1803. 1804.
  1805. 1806. 1807. 1808. 1809. 1810. 1811. 1812. 1813. 1814. 1815. 1816.
  1817. 1818. 1819. 1820. 1821. 1822.]]


(array([[[1.44223245e-07, 4.51734908e-07, 1.82360452e-06, ...,
          6.61006814e-02, 1.00698562e-01, 8.33912859e-02],
         [1.33590507e-07, 4.09861441e-07, 1.69385992e-06, ...,
          1.45927135e-01, 7.50755138e-02, 8.73703979e-02],
         [1.31640558e-07, 4.00899616e-07, 1.66828608e-06, ...,
          9.51339634e-02, 1.28533180e-01, 6.88452684e-02],
         ...,
         [1.60417223e-07, 4.36242519e-07, 1.80457495e-06, ...,
          9.21228521e-02, 5.00330591e-02, 1.18719661e-01],
         [1.87465522e-07, 5.07709069e-07, 2.14712763e-06, ...,
          5.57216373e-02, 6.76845996e-02, 7.57306343e-02],
         [1.53240606e-07, 3.93835448e-07, 1.69164219e-06, ...,
          7.78987884e-02, 4.28245990e-02, 6.44161745e-02]]]), array([[[104.56],
         [103.94],
         [103.51],
         [104.81],
         [105.42],
         [105.16],
         [105.81],
         [105.96],
         [105.98],
         [105.57],
         [106.02],
         [106.88],
         [105.96],
     

In [90]:
n_steps = 30
n_inputs = len(stocks.columns)
n_neurons = 50
n_outputs = 1
learning_rate = 0.0001
batch_size = 1
n_iterations = 50000

# Batch size of 50. 
with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        X_batch, y_batch = next_batch(X_norm, target, batch_size, n_steps)
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if iteration % 1000 == 0:
            mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print(iteration, "\tMSE:", mse)
    y_pred = sess.run(outputs, feed_dict={X: X_batch})
    saver.save(sess, "./stock_p1")

    
        

0 	MSE: 6775.983
1000 	MSE: 2.860226
2000 	MSE: 3477.7698
3000 	MSE: 470.9737
4000 	MSE: 360.66882
5000 	MSE: 570.43134
6000 	MSE: 137.59593
7000 	MSE: 870.5533
8000 	MSE: 94.6551
9000 	MSE: 2130.4744
10000 	MSE: 117.50547
11000 	MSE: 13.164642
12000 	MSE: 1584.3151
13000 	MSE: 1523.7328
14000 	MSE: 331.80368
15000 	MSE: 67.807785
16000 	MSE: 235.74525
17000 	MSE: 370.82623
18000 	MSE: 814.92267
19000 	MSE: 719.9538
20000 	MSE: 918.8492
21000 	MSE: 3882.6304
22000 	MSE: 88.95957
23000 	MSE: 63.283436
24000 	MSE: 2162.7827
25000 	MSE: 30.026548
26000 	MSE: 15.964908
27000 	MSE: 38.997353
28000 	MSE: 537.40924
29000 	MSE: 309.56345
30000 	MSE: 663.8472
31000 	MSE: 53.518726
32000 	MSE: 18.83415
33000 	MSE: 151.18892
34000 	MSE: 11.603282
35000 	MSE: 724.2664
36000 	MSE: 205.19127
37000 	MSE: 10.854187
38000 	MSE: 717.79803
39000 	MSE: 176.65894
40000 	MSE: 60.14807
41000 	MSE: 69.584
42000 	MSE: 577.0596
43000 	MSE: 94.43303
44000 	MSE: 42.351643
45000 	MSE: 66.521385
46000 	MSE: 133.742

# Problem 2. 
Predict price MMM based on the model estimated in Problem 1. Pick 10 best dates to invest in MMM (largest difference between current price of MMM and the price of MMM in 7 days). What is the expected and realized return on this investments?

In [91]:
n_steps = 30
n_inputs = len(stocks.columns)
n_neurons = 50
n_outputs = 1
learning_rate = 0.001
batch_size = 1
n_iterations = 100
lenX = len(X_norm) 
a = (np.zeros(shape=(lenX)))
with tf.Session() as sess:                        # not shown in the book
    saver.restore(sess, "./stock_p1")
    lenst = len(X_norm) - n_steps
    j = 30
    i = 0
    while j < lenX:
        j = i + 30
        sh =  X_norm[i:j, :n_inputs].shape
        sh = int(sh[0])
        tar =  n_steps
        if sh < tar: break
        X_n= X_norm[i:j, :n_inputs].reshape(1, n_steps, n_inputs)
        y_pred = sess.run(outputs, feed_dict={X: X_n})
        y_pred = y_pred.flatten()
        a[i:j] = y_pred
        i = i + 30
    
        
    

INFO:tensorflow:Restoring parameters from ./stock_p1


In [92]:
target.shape

(3012,)

In [93]:
inv = stocks[('High', 'MMM')]
inv.shape

(3020,)

In [94]:
# drop 12 observations dropped when we did not have the last 30 days
y = target[:-12,]
inv = stocks[('High', 'MMM')]
# Investment price, drop first 7 days and last 12 days.
price0 = inv[:-20,]
price7 = target[:-12]
# Predicted y_hat, drop last 12 days as zeros. 
y_hat = a[:-12]

In [95]:
import pandas
dataset = pd.DataFrame({'price0':price0,'price7':y, 'price7_hat':y_hat})

In [96]:
y_hat.shape

(3000,)

In [97]:
dataset["exp_profit"] = 100 * (dataset["price7_hat"] -  dataset["price0"] )/ dataset["price0"]
dataset["profit"] = 100 * (dataset["price7"] -  dataset["price0"] )/ dataset["price0"] 

In [98]:
dataset.nlargest(10, 'exp_profit').mean()

price0         70.024000
price7         72.571000
price7_hat    122.283787
exp_profit     73.656212
profit          5.108711
dtype: float64

In [99]:
dataset["profit"].mean()

0.3085080323132474

In [100]:
dataset["exp_profit"].mean()

1.1752938920301241

In [101]:
dataset.mean()

price0        111.769097
price7        112.148370
price7_hat    110.889058
exp_profit      1.175294
profit          0.308508
dtype: float64

# Answer
For the best 10 days our model predicts average profits of 73.6%, but real average profits over these 10 days are just 5.1%.
Average profit rate for 7 day investment over all periods is 0.3%, while the expected profit is 1.17%. 